In [2]:
import os
from data.source.pg_experiment import get_pg_experiment_dataset, AUDIO_PATH
import polars as pl
from models.SimpleCNN import PronunciationDataset, collate_fn, train, evaluate
from models.SimpleCNN_v1_2 import SimpleCNN
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from path import RESULT_DIRECTORY
import wandb
from dotenv import load_dotenv
from torch.optim.lr_scheduler import ReduceLROnPlateau

load_dotenv()
# Use the API KEY specified in the .env file
wandb.login(key=os.getenv("WANDB_API_KEY"))

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\fisch\_netrc


In [3]:
df_pron, _ = get_pg_experiment_dataset(".wav")

df_stageI_polish = df_pron.filter(pl.col("word_id").str.starts_with("a") & pl.col("mother")
    .str.strip_chars()
    .str.to_lowercase()
    .is_in(["polish", "polski"]))

df_other_lang = df_pron.filter(
    pl.col("word_id").str.starts_with("a") &
    (~pl.col("mother")
       .str.strip_chars()
       .str.to_lowercase()
       .is_in(["polish", "polski"]))
)


In [4]:
count_missing = 0
rec_paths = df_other_lang.select("rec_path").to_series()

for path in rec_paths:
    full_path = os.path.normpath(os.path.join(AUDIO_PATH, path))
    if not os.path.exists(full_path):
        count_missing += 1


count_missing_pl = 0
rec_paths = df_stageI_polish.select("rec_path").to_series()

for path in rec_paths:
    full_path = os.path.normpath(os.path.join(AUDIO_PATH, path))
    if not os.path.exists(full_path):
        count_missing_pl += 1

print(f"Missing files in mother language Polish: {count_missing_pl}")
print(f"Missing files in mother language other than Polish: {count_missing}")

Missing files in mother language Polish: 0
Missing files in mother language other than Polish: 2


In [5]:
# Find indices of rows with missing files
rec_paths = df_other_lang.select("rec_path").to_series()
missing_indices = []

for i, path in enumerate(rec_paths):
    full_path = os.path.normpath(os.path.join(AUDIO_PATH, path))
    if not os.path.exists(full_path):
        missing_indices.append(i)

# Print the deleted rows
deleted_rows = df_other_lang[missing_indices]
print(deleted_rows)

df_other_lang = df_other_lang.filter(
    ~pl.Series(range(len(df_other_lang))).is_in(pl.Series(missing_indices).implode())
)



shape: (2, 7)
┌────────────┬───────┬─────────┬───────────────────┬───────────┬────────┬─────────┐
│ id_student ┆ value ┆ word_id ┆ rec_path          ┆ univ      ┆ gender ┆ mother  │
│ ---        ┆ ---   ┆ ---     ┆ ---               ┆ ---       ┆ ---    ┆ ---     │
│ i64        ┆ i64   ┆ str     ┆ str               ┆ str       ┆ str    ┆ str     │
╞════════════╪═══════╪═════════╪═══════════════════╪═══════════╪════════╪═════════╡
│ 757        ┆ 1     ┆ a0      ┆ stageI/757/a0.wav ┆ CLES_UMK2 ┆ f      ┆ chinese │
│ 757        ┆ 1     ┆ a1      ┆ stageI/757/a1.wav ┆ CLES_UMK2 ┆ f      ┆ chinese │
└────────────┴───────┴─────────┴───────────────────┴───────────┴────────┴─────────┘


In [6]:
import polars as pl
import numpy as np

def stratified_split(df: pl.DataFrame, label_col: str, train_frac=0.8, val_frac=0.1, seed=42):
    classes = df.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []

    rng = np.random.RandomState(seed)

    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)

        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)

        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])
        test_rows.append(class_df[indices[val_end:]])

    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)

    return train_df, val_df, test_df


In [7]:
# dataset = PronunciationDataset(df_stageI_polish, base_dir=AUDIO_PATH)
dataset_other = PronunciationDataset(df_other_lang ,base_dir=AUDIO_PATH)

In [8]:
train_pl, val_pl, test_pl = stratified_split(df_stageI_polish, label_col="value", train_frac=0.6, val_frac=0.2)

dataset_train = PronunciationDataset(train_pl, base_dir=AUDIO_PATH)
dataset_val = PronunciationDataset(val_pl, base_dir=AUDIO_PATH)
dataset_test = PronunciationDataset(test_pl, base_dir=AUDIO_PATH)


In [10]:
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lr=1e-3
pth = "SimpleCNN_v1_2.pth"
epochs = 10
weight_decay = 1e-4

# Start a new wandb run to track this script.
run = wandb.init(
    # name of the team
    entity="fischbach-kamil-pg",
    project="mandarin-pronunciation",
    # name of the run
    name="SimpleCNN-v1-2-run-1",
    config={
        "Name": "SimpleCNN_v1-2",
        "learning_rate": lr ,
        "optimizer": "Adam",
        "criterion": "BCELoss",
        "architecture": "SimpleCNN_v1-2",
        "dataset": "Stage-I-only-polish",
        "train_val_test(%)": "60-20-20",
        "epochs": epochs,
        "scheduler": "ReduceLROnPlateau"
    },
)

model = SimpleCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.BCELoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    scheduler.step(val_loss)
    # Logging the metadata for each epoch so that the charts can be generated on the dashboard
    run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

run.log({"model_eval": model.eval()})
# Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb so we should be fine.
torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, pth))
artifact = wandb.Artifact("simplecnn-model", type="model")
artifact.add_file(os.path.join(RESULT_DIRECTORY, pth))
run.log_artifact(artifact)

# Finish the run so it gets send to the remote. You can discover the run right after that on the dashboard.
run.finish()


Epoch 1, Train Loss: 0.8441, Train Acc: 0.5342, Val Loss: 0.6715, Val Acc: 0.5838
Epoch 2, Train Loss: 0.6706, Train Acc: 0.5568, Val Loss: 0.6619, Val Acc: 0.5995
Epoch 3, Train Loss: 0.6621, Train Acc: 0.5990, Val Loss: 0.6506, Val Acc: 0.5995
Epoch 4, Train Loss: 0.6612, Train Acc: 0.5967, Val Loss: 0.6639, Val Acc: 0.6030
Epoch 5, Train Loss: 0.6539, Train Acc: 0.6031, Val Loss: 0.6535, Val Acc: 0.5986
Epoch 6, Train Loss: 0.6562, Train Acc: 0.5996, Val Loss: 0.6494, Val Acc: 0.5986
Epoch 7, Train Loss: 0.6535, Train Acc: 0.6008, Val Loss: 0.6492, Val Acc: 0.6030
Epoch 8, Train Loss: 0.6462, Train Acc: 0.6013, Val Loss: 0.6438, Val Acc: 0.5995
Epoch 9, Train Loss: 0.6457, Train Acc: 0.5984, Val Loss: 0.6510, Val Acc: 0.6003
Epoch 10, Train Loss: 0.6447, Train Acc: 0.6031, Val Loss: 0.6407, Val Acc: 0.6126


train_acc,▁▃█▇██████
train_loss,█▂▂▂▁▁▁▁▁▁
val_acc,▁▅▅▆▅▅▆▅▅█
val_loss,█▆▃▆▄▃▃▂▃▁
model_eval,SimpleCNN( (conv1)...
train_acc,0.60308
train_loss,0.64469
val_acc,0.61257
val_loss,0.64067
